#  Open Information Extraction

In this lab you will be implementing an *open information extraction* system, almost entirely from scratch. Information extraction takes a body of freeform text and extracts the contained information in a computer interpretable form. The word *open* simply means that the text/facts are arbitrary, so it will work with any input rather than a specific domain (e.g. legal texts).

As an example, given the input:

> "Trolls really don't like the sun."
  

you may extract the "fact":
```
('Trolls', 'do not like', 'the sun')
```

The approach is based on the paper "*Identifying Relations for Open Information Extraction*", by Fader, Soderland & Etzioni. Parts have been updated with more recent, or more ML, techniques however (Q3 and Q4 match the paper; Q1 and Q2 don't). You don't have to read the paper as this workbook takes you through the process; this lab is also intended as a tutorial/opportunity to see one way such a system works. Some parts are rule based, as that's still often the case. Note that the several parts of a complete system have been dropped, as the lab would be too much work otherwise; consequentially it isn't going to work that well.

The steps of the system are as follows:
*  Tokenise and split on sentences *(provided)*


1. Part of speech tagging - token level
2. Part of speech tagging - sentence level
3. Named entity resolution
4. Relation extraction


*  Summarise "*20,000 leagues under the seas*" by Jules Verne *(provided)*

A simple NLP library, called `ogonek`, is provided. It has some basic functionality that you will require; this is for two reasons:
1. This has to run on BUCS computers, and normal NLP libraries do not.
2. To keep certain low level functionality consistent, so automarking is easier.

Its documentation can be found below in a markdown cell.

## Ogonek

A tiny NLP library, that contains exactly the functionality I don't want you to implement for this coursework!



### Tokenisation and sentence splitting
`ogonek.Tokenise()`

A class that tokenises some text and splits it into sentences. Construct an instance with `tokens = ogonek.Tokenise('My text')`; it then has the same interface as a list of lists:
* `len(tokens)`: Number of extracted sentences (not words)
* `tokens[i]`: Sentence i, where i ranges from 0 to one less than `len(tokens)`. A sentence is a list of tokens.



### Word vectors
`ogonek.Glove()`

Constructing a `glove = ogonek.Glove()` object loads a heavily pruned Glove word vectors from the file `baby_glove.zip` into memory, and will then translate tokens into word vectors. Note that it automatically lowercases any token it is handed, so you don't need to. Has the following interface:
* `glove.len_vec()` - Returns the length of the word vectors; should be 300.
* `len(glove)` - Returns how many word vectors it knows of.
* `token in glove` - Returns `True` if it has a word vector for that token, `False` otherwise.
* `glove[token]` - Returns the word vector for the given token; raises an error if it does not have one.
* `glove.decode(token)` - Returns the word vector for the given token, but if the word vector is unknown returns a vector of zeros instead (silent failure).
* `glove.decodes(list of tokens)` - Returns a list of word vectors, one for each token. Has the same silent failure behaviour as `decode`.



### Groningen Meaning Bank dataset
`ogonek.GMB()`

Provides access to the Groningen Meaning Bank dataset, which is supplied in the file `ner_dataset.csv`. Replicates the interface of the tokenisation system as far as it can. Construct with `gmb = ogonek.GMB()`; has the following interface:
* `len(gmb)`: Number of sentences (not words) in data set
* `gmb[i]`: Sentence i, where i ranges from 0 to one less than `len(gmb)`. A sentence is a list of tokens.
* `gmb.pos(i)`: A list of POS tags that match with sentence i. Note that these are the full Penn Treebank tags (not the reduced set used below).
* `gmb.ner(i)`: A list of named entities that match with sentence i. Using outside-inside scheme.



### Pretty printing

`ogonek.aligned_print(*)` takes multiple lists and prints them out, aligning them so that all elements in position 0 of all lists are aligned vertically (extra space added as required), and then elements in position 1 and so on. For showing tags and a sentence with everything aligned. Also does word wrap and colour coding.

In [1]:
%matplotlib inline

import time
import string
import re

import numpy
import matplotlib.pyplot as plt

import ogonek


## Useful variables

In [2]:
# Dictionary giving descriptions of the reduced part of speech tags...
rpos_desc = {'C' : 'Coordinating conjunction',
             '0' : 'Cardinal number',
             'D' : 'Determiner',
             'E' : 'Existential there',
             'I' : 'Preposition or subordinating conjunction',
             'J' : 'Adjective',
             'N' : 'Noun',
             'P' : 'Predeterminer',
             'S' : 'Possessive ending',
             'M' : 'Pronoun',
             'R' : 'Adverb',
             'Z' : 'Particle',
             'T' : 'to',
             'V' : 'Verb',
             'A' : 'Anything else',
             '.' : 'All punctuation'}



# Reduced list of part of speech tags as a list...
num_to_rpos = ['C', '0', 'D', 'E', 'I', 'J', 'N', 'P',
               'S', 'M', 'R', 'Z', 'T', 'V', 'A', '.']



# Dictionary that maps a reduced part of speech
# tag to it's index in the above list; useful for vectors/matrices etc...
rpos_to_num = {'C' : 0,
               '0' : 1,
               'D' : 2,
               'E' : 3,
               'I' : 4,
               'J' : 5,
               'N' : 6,
               'P' : 7,
               'S' : 8,
               'M' : 9,
               'R' : 10,
               'Z' : 11,
               'T' : 12,
               'V' : 13,
               'A' : 14,
               '.' : 15}



# Dictionary that maps the full part of speech tags to the reduced set...
pos_to_rpos = {'CC' : 'C',
               'CD' : '0',
               'DT' : 'D',
               'EX' : 'E',
               'FW' : 'A',
               'IN' : 'I',
               'JJ' : 'J',
               'JJR' : 'J',
               'JJS' : 'J',
               'LS' : 'A',
               'MD' : 'A',
               'NN' : 'N',
               'NNS' : 'N',
               'NNP' : 'N',
               'NNPS' : 'N',
               'PDT' : 'P',
               'POS' : 'S',
               'PRP' : 'M',
               'PRP$' : 'M',
               'RB' : 'R',
               'RBR' : 'R',
               'RBS' : 'R',
               'RP' : 'Z',
               'SYM' : 'A',
               'TO' : 'T',
               'UH' : 'A',
               'VB' : 'V',
               'VBD' : 'V',
               'VBG' : 'V',
               'VBN' : 'V',
               'VBP' : 'V',
               'VBZ' : 'V',
               'WDT' : 'D',
               'WP' : 'M',
               'WP$' : 'S',
               'WRB' : 'R',
               '-' : '.',
               'LRB' : '.',
               'RRB' : '.',
               '``' : '.',
               '"' : '.',
               '.' : '.',
               ',' : '.',
               ';' : '.',
               ':' : '.',
               '$' : '.'}


## Load book, tokenise and split on sentences
The below code reads in the book, chops it down to just the text of the book, and then tokenises it using the provided `ogonek` library.

In [1]:
# Loop file, only keeping lines between indicators...
lines = []
record = False

with open('20,000 Leagues Under the Seas.txt', 'r') as fin:
    for line in fin:
        if record:
            if line.startswith('***END OF THE PROJECT GUTENBERG'):
                break
      
            lines.append(line)
    
        else:
            if line.startswith('***START OF THE PROJECT GUTENBERG'):
                record = True

text = ''.join(lines)


# Tokenise...
under_the_seas = ogonek.Tokenise(text)


# Print 10 random sentences to check it worked...
numpy.random.seed(0)

for i in range(10):
    toks = numpy.random.choice(under_the_seas)
    print('{:02d}. {}'.format(i+1, ' '.join(toks)))


# 1. Part of speech tagging - token level

The goal here is to train a classifier that indicates which of the part of speech tags (the reduced set provided above) each word is. For this initial approach you're going to treat words (tokens) individually, without context. For features the Glove word vectors are going to be used (provided by `ogonek.Glove()`).

Instead of training a single classifier a slight modification of a random kitchen sink for each part of speech tag is going to be used. Specifically, a logistic random kitchen sink that indicates the probability that the word should be labelled with the associated tag. This is a *one vs all* classifier - you have a classifier for every tag, run them all on each word, and then select the tag with the highest probability (it's inconsistent - they won't sum to 1!). A logistic random kitchen sink is simply a normal kitchen sink that is pushed through a sigmoid function (in neural network terms, the final layer has a non-linearity),
$$\operatorname{Sig}(z) = \frac{1}{1 + e^{-z}}$$
such that the final binary classifier is
$$P(\textrm{tag}) = \operatorname{Sig}\left(\sum_{k \in K} \alpha_k \phi\left(\vec{x} \cdot \vec{w}_k\right)\right)$$
For the cost function you should maximise the log likelihood of the dataset. This will require gradient descent; Nestorov or better, including backtracking line search to select the initial step size, to get all marks (it will be very slow otherwise). You can either differentiate yourself, copy the equations from lecture 7 of ML1 (where they were derived), or use tensor flow; your choice! The non-linearity, $\phi(\cdot)$ is up to you ($\sin$ works). Remember to include the original features plus the value `1` when creating the extended feature vector (so it has a bias term). It is suggested to use 300 random features, in addition to the 300 provided by glove (total of 601 - bias term is the +1), as that keeps the resulting data matrix during training small enough that it completes reasonably quickly.

The Groningen Meaning Bank dataset has been provided; it can be accessed via the class `ogonek.GMB`. It includes lots of sentences, each as a list of tokens, plus part of speech tags as a list aligned with the sentence.
Source: https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus


In [4]:
# Load word vectors; in a seperate cell as this takes a couple seconds...
glove = ogonek.Glove()


# Groningen Meaning Bank dataset - a set of sentences each tagged
# with part of speech and named entity recognitiuon tags...
gmb = ogonek.GMB()
print('GMB sentences = {}'.format(len(gmb)))
print()


# Print out 5 random sentences from GMB with POS and NER tags, to illustrate the data...
numpy.random.seed(1)
for _ in range(5):
    i = numpy.random.randint(len(gmb))
    ogonek.aligned_print(gmb[i], gmb.pos(i), gmb.ner(i))


GMB sentences = 47959

The U.S.  space agency is  making final preparations to launch the first 
DT  NNP   NN    NN     VBZ VBG    JJ    NNS          TO VB     DT  JJ    
O   B-geo O     O      O   O      O     O            O  O      O   O     

direct space probe to the distant planet of Pluto . 
JJ     NN    NN    TO DT  JJ      NN     IN NNP   . 
O      O     O     O  O   O       O      O  B-geo O 

On Monday , the freighter Torgelow was hijacked off the eastern coast of 
IN NNP    , DT  NN        NNP      VBD VBN      IN  DT  JJ      NN    IN 
O  B-tim  O O   O         B-art    O   O        O   O   O       O     O  

Somalia . 
NNP     . 
B-geo   O 

Chile and Bolivia are associate members . 
NNP   CC  NNP     VBP JJ        NNS     . 
B-gpe O   B-gpe   O   O         O       O 

Venezuela has freed 11 Colombian soldiers who had been detained after entering 
NNP       VBZ VBN   CD JJ        NNS      WP  VBD VBN  VBN      IN    VBG      
B-geo     O   O     O  B-gpe     O        O   O

In [37]:
def sigmoid(x):
    """This function returns the sigmoid
    value of a single number or array of
    numbers.
        
    ARGUMENTS
    ---------
    x : number or array of numbers
        
    RETURNS
    -------
    sigmoid value
    """
    return (1/(1+numpy.exp(-x)))  


def alpha_gradient(y_train, alpha, ex):
    """This function calculates and 
    returns the gradient value of alpha
    at every instance.
    
    ARGUMENTS
    ---------
    y_train : actual tags
    alpha : current alpha value set
    ex : the word extended feature set
    
    RETURNS
    -------
    grad : alpha gradient calculated as (y_train - ex @ alpha) @ ex
    """
    grad = numpy.zeros((601,1))                    # initialize gradient as the same size as alpha
    p = y_train
    q = sigmoid(ex.dot(alpha))                     # y_predictions

    grad += numpy.einsum('ij,ik->kj',(p-q), ex)    # enisum returns the transposed dot product
    return grad
    

def log_likelihood(y_train, alpha, ex):
    """This function returns the calculated 
    log likelihood at every instance. 
    
    ARGUMENTS
    ---------
    y_train : actual tags
    alpha : current alpha value set
    ex : the word extended feature set
    
    RETURNS
    -------
    ll : log likelihood calculated as Summation(y_train*log(y_pred) + (1-y_train)*log(1-y_pred))
    """
    p = y_train
    q = sigmoid(ex.dot(alpha))                        # y_predictions
    q = numpy.clip(q, 1e-3, 1 - 1e-3)                 # clipping the prediction value so as to avoid Nan or inf value while taking log      
    
    ll = sum(p*numpy.log(q) + (1-p)*numpy.log(1-q))   # log likelihood expression
    
    return ll

def backtracking(y_train, alpha, ex):
    """This function calculates and returns
    the best step size for given set of 
    parameters. It is calculated using the 
    Armijo-Goldstein condition.
    
    ARGUMENTS
    ---------
    y_train : actual tags
    alpha : current alpha value set
    ex : the word extended feature set
    
    RETURNS
    -------
    ita : best step size 
    """
    ita = 1.0                  # initial step size
    mu = 0.5                   # mu value from lecture

    # Armijo-Goldstein condition: loss(alpha + ita*alpha_gradient) > loss(alpha) - ita*mu*2nd_order_matrix_norm(alpha_gradient)
    while (log_likelihood(y_train, alpha + ita*alpha_gradient(y_train, alpha, ex), ex)) > (log_likelihood(y_train, alpha, ex) + ita*mu*numpy.linalg.norm(alpha_gradient(y_train, alpha, ex), ord=2)):
        ita *= 0.8             # reduce step size by 0.8 factor at every iteration
        if ita < 1e-6:         # if ita gets too low break and use the last updated ita value
            break
                                            
    return ita

def nestrov(y_train, initial_alpha, ex, ita):
    """This function returns the optimized alpha
    values using Nestrov gradient descent. It is
    obtained by maximizing the log likelihood.
    
    ARGUMENTS
    ---------
    y_train : actual tags
    alpha : current alpha value set
    ex : the word extended feature set
    ita : best step size
    
    RETURNS
    -------
    alpha_new : best alpha value with maximum log likelihood
    best_ll : best log likelihood
    """
    iter_lim = 256
    lamda = 0.8
    phi = numpy.zeros((601,1))                              # initialize phi as same size as alpha
    alpha = initial_alpha
    iteration = 0                                           # iteration counter
    alpha = alpha - lamda*phi                               # initial change in alpha 
    grad = alpha_gradient(y_train, alpha, ex)               # initial alpha gradient
    best_ll = log_likelihood(y_train, alpha, ex)            # initial log likelihood
    
    for itr in range(iter_lim):                             # the loop runs for 256 iterations unless terminated earlier
        
        phi_new = lamda*phi + ita*grad                      # update phi gradient
        alpha_new = alpha + phi_new                         # update alpha
        
        grad_new = alpha_gradient(y_train, alpha_new, ex)   # calculate new alpha gradient with updated alpha
        new_ll = log_likelihood(y_train, alpha_new, ex)     # calculate new log likelihood with udated alpha
        
        if(itr%10==0):     
            print('iteration: ',itr, 'loss:', new_ll)       # Prints log likelihood at every 10 iterations
        
        if new_ll > best_ll:                                # check whether new ll is better than best ll
            best_ll = new_ll                                # update best ll
            grad = grad_new                                 # update to corresponding gradient
            alpha = alpha_new                               # update to corresponding alpha
            phi = phi_new                                   # update to corresponding phi
            iteration = 0                    
        else:
            iteration += 1                                  # if no improvement, count iterations with no improvements
            if iteration > 8:                               # break out of for loop if no improvement uptil 8 iterations
                break
            
    return alpha_new, best_ll

def postag_to_y_value(tag, sentence_id):
    """This function returns the y_train 
    value for a given tag and sentence. If
    tag is true returns 1 else returns 0.
    
    ARGUMENTS
    ---------
    tag: pos tag
    sentence_id : gmb sentence number id
    
    RETURNS
    -------
    all_y : all y_train values for the sentence as a list
    """
    pos_extended_tags = gmb.pos(sentence_id)        # get extended pos tags
    pos_reduced_tags = []
    all_y = []
    for ex_tag in pos_extended_tags:
        new_tag = pos_to_rpos[ex_tag]               # convert extended pos tags to rpos
        
        if new_tag==tag:
            all_y.append(1)                         # for true tag, returns 1
        else:
            all_y.append(0)                         # for false tag, return 0
    
    return all_y

In [38]:
# A test/train split - train with [0:split], test with [split:len(gmb)]
split = int(len(gmb) * 0.3) # Have a lot of data, and don't want you waiting around too long to train!
print('Using {} sentences for training'.format(split))
x_temp = gmb[0:split]

# Preparing the x_train list with all the sentence tokens together
x_train = []
for sntc in x_temp:           
    for word in sntc:
        x_train.append(word)
        

def train_tag_model(tag):
    start = time.time()
    

    # Preparing y_train as a list of 0s and 1s for current tag 
    y_train = []
    for i in range(len(x_temp)):
        y_tag = postag_to_y_value(tag, i)
        for j in y_tag:
            y_train.append(j)
    y_train = numpy.array(y_train)
    
    
    #************* LOGISTIC RANDOM KITCHEN SINK IMPLEMENTATION ***************#
    
    x = numpy.array(glove.decodes(x_train))                         # exracting 300 features for all words  
    w_k = numpy.random.normal(loc=0.0, scale=1.0, size=(300,300))   # random initialization of w_k
    nf = numpy.sin(numpy.einsum('ef,gf->eg',x,w_k))                 # sin(x @ w_k)
            
    ex = numpy.append(x, nf, axis=1)                                # extending word features to 600
    ones = numpy.ones((x.shape[0],1))                               
            
    ex = numpy.append(ex, ones, axis=1)                             # adding bias making it 601 features

    alpha = numpy.random.normal(loc=0.0, scale=1.0, size=(601,1))   # random initialization of alpha
#     ita = backtracking(y_train.reshape((313964, 1)), alpha, ex)
    ita = 0.01      # Did not use backtracking as it did not improve the test accuracy but it has been fully implemented
    updated_alpha, best_ll = nestrov(y_train.reshape((313964, 1)), alpha, ex, ita)  # finding best alpha
    print('Log Likelihood: ', best_ll)
    
    
    end = time.time()
    print('  (took {:g} seconds)'.format(end-start))
    return updated_alpha, w_k                                       # return updated alpha and corresponding weights



# Code to train a model for each reduced POS tag...
rpos_model = {}
for tag in rpos_desc:
    print('Training {}'.format(tag))
    rpos_model[tag] = train_tag_model(tag)


Using 14387 sentences for training
Training C
iteration:  0 loss: [-70424.60384458]
iteration:  10 loss: [-27195.22309082]
Log Likelihood:  [-27166.3662656]
  (took 26.2993 seconds)
Training 0
iteration:  0 loss: [-64077.57892017]
iteration:  10 loss: [-41428.76240506]
iteration:  20 loss: [-39009.3304513]
Log Likelihood:  [-29986.45235023]
  (took 28.7282 seconds)
Training D
iteration:  0 loss: [-100617.91248853]
iteration:  10 loss: [-52948.05978461]
Log Likelihood:  [-52843.46476118]
  (took 24.726 seconds)
Training E
iteration:  0 loss: [-22886.67070605]
iteration:  10 loss: [-20783.7123676]
iteration:  20 loss: [-18596.3009858]
iteration:  30 loss: [-16945.58659366]
iteration:  40 loss: [-14936.62059014]
iteration:  50 loss: [-13443.86192088]
iteration:  60 loss: [-12243.38201377]
iteration:  70 loss: [-11088.89421888]
iteration:  80 loss: [-9943.36721975]
iteration:  90 loss: [-9023.53886256]
iteration:  100 loss: [-8236.1688178]
iteration:  110 loss: [-7690.53519029]
iteration: 

In [52]:
def predict(x, tag):
    """This function returns a probability
    value for each tag for a given word.
    
    ARGUMENTS
    ---------
    x : array of 300 word features
    tag : pos tag
    
    RETURNS
    -------
    y_pred : probability of word having the current tag
    """
    
    # Implementing logistic random kitchen sink for single word
    
    w_k = rpos_model[tag][1]
           
    nf = numpy.sin(numpy.einsum('f,gf->g',x,w_k))
  
    ex = numpy.append(x, nf, axis=0)
    ones = numpy.ones((x.shape[0],1))

    ex = numpy.append(ex, numpy.ones(1), axis=0)
    
    alpha = rpos_model[tag][0]
    
    temp = ex.dot(alpha)            
    y_pred = sigmoid(temp)        # prediction proability is sigmoid(ex @ alpha)
    
    return y_pred
    
# You will want to train the models above, then fill in the below function to estimate POS tags...
def token_pos(sentence):
    """Given a sentence, as a list of tokens, this should return part of
    speech tags, as a list of strings (the codes in the rpos_desc dictionary).
    Basically calls the models for each tag and selects the tag with the
    highest probability."""
    
    # **************************************************************** some of the above marks
    # argmax on all probablities in the array og=f 16 probabilities
    pred_tag = []
    for token in sentence:  
        x = glove.decode(token)                 # extracting 300 word features 
        
        preds =[]
        for tag in rpos_desc:
            prediction = predict(x, tag)
            preds.append(prediction)            # list containing probability of each tag for a token
         
        pred_ind = numpy.argmax(preds)          # pas with max probability is the token pos
        pred_tag.append(num_to_rpos[pred_ind])
        
    return pred_tag                             # list of predicted tags for each token in sentence



# Code to test the performance of your POS tagger...
correct = 0
tested = 0
pershown = 0
stop_percent = 100 # If you want faster feedback you can reduce this

start = time.time()
for i in range(split, len(gmb)):
    percent = int(100 * (i - split) / (len(gmb) - split))
    if percent>pershown:
        pershown = percent
        print('\r{: 3d}%'.format(percent), end='')
    
    if percent>=stop_percent:
        break
    
    guess = token_pos(gmb[i])
    truth = gmb.pos(i)
    
    for g,t in zip(guess, truth):
        if g==pos_to_rpos[t]:
            correct += 1
        tested += 1
end = time.time()

print()
print('Percentage correct = {:.1f}%'.format(100 * correct / tested))
print('  (took {:g} seconds)'.format(end - start))


 99%
Percentage correct = 56.2%
  (took 1141.07 seconds)


# 2. Part of speech tagging - sentence level

While the previous step works very well you need POS tags to be super accurate, as everything else depends on them. You will now introduce context. This is done by calculating transition probabilities between tags and solving a Markov random chain using the forward-backwards algorithm (or just forward if you keep links; it's dynamic programming) to find the maximum a posteriori (MAP) POS tag assignment for the entire sentence. The adjacency matrix should contain $\log P(\textrm{second pos tag} | \textrm{first pos tag})$. lec 11 term 2


In [56]:
# Adjacent Probabilities

adj_prob = numpy.ones((16,16))        # initializing all as ones as we are going to work in log space
x_temp = gmb[0:split]                 # calculating probabilities based on training set only
 
    
for i in range(len(x_temp)):          # for every sentence in training data
    ext_tags = gmb.pos(i)             # get pos of tokens
    tag_ind = []
    for ex_tag in ext_tags:
        ind = rpos_to_num[pos_to_rpos[ex_tag]]    # convert extended tags to rpos
        tag_ind.append(ind)
        
    for j in range(len(tag_ind)-1):
        adj_prob[tag_ind[j]][tag_ind[j+1]] += 1   # check the pair and update count accordingly in adj_prob matrix
            
log_adj_probs = numpy.log(adj_prob)               # taking log

max_probs = numpy.amax(log_adj_probs, axis=0)     # finding max for normalization along first tag

norm_adj_probs = []
for i in range(16):
    norm_prob = log_adj_probs[i]/max_probs[i]     # normalization
    norm_adj_probs.append(norm_prob)  

    
print(norm_adj_probs)                 # normalized adjacent probabilities in log space
    
    

[array([0.        , 0.66247107, 0.7991075 , 0.32963192, 0.66110712,
       0.83313072, 0.92514595, 0.        , 0.        , 0.67531336,
       0.64959269, 0.        , 0.46024548, 0.87616876, 0.51330437,
       0.44990073]), array([0.6507813 , 0.75947945, 0.53535445, 0.        , 0.83328178,
       0.86739916, 1.07018286, 0.        , 0.        , 0.23333881,
       0.49281073, 0.        , 0.5944099 , 0.67554738, 0.28614207,
       0.91936601]), array([0.22105121, 0.68637504, 0.43163794, 0.07368374, 0.52300953,
       0.95200618, 1.05471547, 0.        , 0.07368374, 0.47715624,
       0.5808226 , 0.17108834, 0.28054014, 0.76277725, 0.5182489 ,
       0.53816987]), array([0.15957142, 0.        , 0.15957142, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.37051335, 0.        , 0.        , 1.2162616 , 0.5904844 ,
       0.15957142]), array([0.34307544, 0.76715602, 0.9398202 , 0.31325425, 0.62479414,
       0.83321273, 0.94709087, 0.27055008, 0

In [57]:
# Emission Probabilities
# adj_prob: transition probability from tag 1 to tag 2
# tag_prob: probability of tag 1

# emm = adj_prob/tag_prob

tag_prob = numpy.ones(16)

# counting occurances of each tag

for i in range(len(x_temp)):
    ext_tags = gmb.pos(i)
    tag_ind = []
    for ex_tag in ext_tags:
        ind = rpos_to_num[pos_to_rpos[ex_tag]]
        tag_ind.append(ind)
        
    for j in range(len(tag_ind)):
        tag_prob[tag_ind[j]] += 1
               

# calculating emission probabilities

emm_probs = []
for i in range(16):
    emm_prob = adj_prob[i]/tag_prob[i]
    emm_probs.append(emm_prob)

# converting to log space     
log_emm_probs = numpy.log(emm_probs)    

# normalization

max_emm_probs = -1*(numpy.amax((-1*log_emm_probs), axis=0))

norm_emm_probs = []
for i in range(16):
    norm_prob = log_emm_probs[i]/max_emm_probs[i]
    norm_emm_probs.append(norm_prob)
    
print(norm_emm_probs) # normalized log emission probabilities

[array([1.        , 0.37206308, 0.24254942, 0.68755156, 0.37335593,
       0.21029981, 0.12308127, 1.        , 1.        , 0.35989026,
       0.38427013, 1.        , 0.5637468 , 0.16950531, 0.51345384,
       0.57355228]), array([0.59079836, 0.4635179 , 0.72595779, 1.35283281, 0.37709883,
       0.33714897, 0.09969874, 1.35283281, 1.35283281, 1.07960398,
       0.7757745 , 1.35283281, 0.65680663, 0.56179843, 1.01777383,
       0.27629822]), array([1.17609248, 0.55274337, 0.89398991, 1.37350637, 0.77158826,
       0.19690314, 0.05931348, 1.47221332, 1.37350637, 0.83301346,
       0.69414171, 1.24302289, 1.09640095, 0.45039474, 0.77796563,
       0.75127944]), array([0.46206713, 0.530472  , 0.46206713, 0.530472  , 0.530472  ,
       0.530472  , 0.530472  , 0.530472  , 0.530472  , 0.530472  ,
       0.37164083, 0.530472  , 0.530472  , 0.00908661, 0.27734393,
       0.46206713]), array([0.95065217, 0.37917249, 0.14649484, 0.99083842, 0.57101556,
       0.29015622, 0.13669708, 1.04838541, 1

In [68]:
# Viterbi

def viterbi(words, tags, s_p, t_p, e_p):
    """This function implements the viterbi algorithm.
    
    ARGUMENTS
    ---------
    words : tokens in a given sentence
    tags : list of all pos tags
    s_p : start probabilities of each tag for each word
    t_p : transition probabilities 
    e_p : emission probabilities
    
    RETURNS
    -------
    x : list of predicted tokens
    """
    x = ['N']*len(words)         # Initialize the prediction list
    
    # As the algorithm encounter a number of errors which were not completely resolved
    # in time, it has not been used. However, a complete implementation and the throught-
    # process behind it has been explained in detail
    
    using_viterbi = 0
    if using_viterbi:
        x = ['N']*len(words)
        T1 = [[0] * len(tags)]                  # probaility list
        T2 = [[None] * len(tags)]               # tag list
        
        for j in range(len(tags)):              # Looping over each tag and initializing the first values with start probabilities
            T1[0][j] = s_p[0][j] + e_p[j][0]    # T1[first] = start_prob * emission_prob
        
        # Looping over every word and tag
        for i in range(1,len(words)-1):          
            for j in range(len(tags)-1):
                # For given word and tag, we find the T1[word][tag] which is maximum
                
                # Initializing with first tag values
                # adding everything up instead of multiplying as we are working in log space
                T1[i][j] = T1[i-1][j] + t_p[j][j] + e_p[j][i]  
                T2[i][j] = T1[i-1][j] + t_p[j][j]
            
                # Looping over the rest of the tags to find the one with maximum T1[word][tag] value
                for k in range(j, len(tags)-j):
                    temp_T1 = T1[i-1][k] + t_p[k][j] + e_p[j][i]
                    temp_T2 = T1[i-1][k] + t_p[k][j]
                
                    if temp_T1 > T1[i][j]:  # update if new calculated T1 is greater than best T1
                        T1[i][j] = temp_T1
                        T2[i][j] = numpy.where(T2 == temp_T2)  # update the corresponding tag in T2
     
    
        z = [0]*len(words)     # stores the best T1 for each tag
    
        z[len(words)] = T1[0][len(words)]    # best T1 for each tag is the first element of the T1 list
        x[len(words)] = tags[z[len(words)]]  # best probable tag index is the corresponding index
    
        # Looping over tags to find the best path 
        for k in range(1,len(tags)):
            # For given word and tag, we find the T1[word][tag] which is maximum
            
            # Initialize the temps
            temp_z = T1[k][len(words)]
            temp_x = tags[z[len(words)]]
        
            # If new z is better than best z we update z and x
            if temp_z > z[len(words)]:
                z[len(words)] = temp_z
                x[len(words)] = temp_x
    
        # Finally we complete the path for each word in reverse
        for i in range(len(words), 2):
            z[i-1] = T2[z[i]][i]
            x[i-1] = tags[z[i-1]]
        
    return x    # return a list of predicted tags


In [2]:

def sentence_pos(sentence):
    """Given a sentence, as a list of tokens, this should return part of
    speech tags, as a list of strings (the codes in the rpos_desc dictionary).
    A more advanced version of token_pos that uses neighbours as well."""
    
    words = sentence
    tags = num_to_rpos
    trans = norm_adj_probs     # size(16,16)
    emm = norm_emm_probs       # size(16,16)
    
    # we find start probabilities value from token_pos and take log
    start_p = []
    for token in sentence:  
        x = glove.decode(token) 
        preds =[]
        for tag in rpos_desc:
            prediction = numpy.log(predict(x, tag)+1)  # adding 1 to avoid encountering log(0)
            preds.append(prediction)
        start_p.append(preds)
        
    all_tags = viterbi(words, tags, start_p, trans, emm)

    return all_tags

sentence = gmb[0]
sentence_pos(sentence)

In [51]:
# Code to test the performance of your improved POS tagger...
correct = 0
tested = 0
pershown = 0
stop_percent = 100 # If you want faster feedback you can reduce this

start = time.time()
for i in range(split, len(gmb)):
    percent = int(100 * (i - split) / (len(gmb) - split))
    if percent>pershown:
        pershown = percent
        print('\r{: 3d}%'.format(percent), end='')
    
    if percent>=stop_percent:
        break
    
    guess = sentence_pos(gmb[i])
    truth = gmb.pos(i)
    
    for g,t in zip(guess, truth):
        if g==pos_to_rpos[t]:
            correct += 1
        tested += 1
end = time.time()

print()
print('Percentage correct = {:.1f}%'.format(100 * correct / tested))
print('  (took {:g} seconds)'.format(end - start))


 99%
Percentage correct = 33.9%
  (took 0.281279 seconds)


# 3. Named entity recognition

The next step is to identify names, that is the entities that "facts" may apply to. While training a further classifier does work (same as above, inc. dynamic programming) there would be little point in repeating the exercise. Instead, a simple rule based approach using *regular expressions* is going to be used.

You will probably want to look at the Python 3 documentation:
https://docs.python.org/3/library/re.html
There is also the how to, a tutorial:
https://docs.python.org/3/howto/regex.html


Given part of speech tagging a name can be defined as:
* An optional *determiner*, e.g. *the* (1 or none)
* An arbitrary number of *adjectives* (could be none)
* A single *noun*

Convert this into a regular expression and finish the function `sentence_ner()` below.


In [66]:
def sentence_ner(sentence, pos):
    """Given a sentence as a list of tokens and it's part of speech tags
    this returns a list of the same length with True wherever it thinks
    there is a name."""
    
    pos_str = ''.join(pos)
    
    pattern = 'D*J*N+'    #RegEx pattern for optional determiner + arbitrary adjectives + single noun
    tag_true = [p for p in re.finditer(pattern, pos_str)]   # find index where true
    
    ret = [False] * len(sentence)
    
    for i in range(len(sentence)):
        if i in tag_true:
            ret[i] = True
    
    # **************************************************************** 2 marks
    
    return ret     # list where presence of noun is expressed as True


In [67]:
# Code to test the performance of the NER tagger...
correct = 0
tested = 0
pershown = 0
stop_percent = 100 # If you want faster feedback you can reduce this

start = time.time()
for i in range(split, len(gmb)):
    percent = int(100 * (i - split) / (len(gmb) - split))
    if percent>pershown:
        pershown = percent
        print('\r{: 3d}%'.format(percent), end='')
    
    if percent>=stop_percent:
        break
    
    guess = sentence_ner(gmb[i], [pos_to_rpos[p] for p in gmb.pos(i)])
    truth = [ner!='O' for ner in gmb.ner(i)]
    
    for g,t in zip(guess, truth):
        if g==t:
            correct += 1
        tested += 1
end = time.time()

print()
print('Percentage correct = {:.1f}%'.format(100 * correct / tested))
print('  (took {:g} seconds)'.format(end - start))


 99%
Percentage correct = 84.7%
  (took 0.619673 seconds)


# 4. Relation extraction

This is where the paper "*Identifying Relations for Open Information Extraction*" comes in, specifically one of its novel contributions. It extracts relations using this procedure:
1. Find relation text by matching a human-designed pattern to the POS tags
2. Identify the named entities to the left and right of the relation text.
3. Generate the relation tuple (left named entity, relation text, right named entity)

(all previous approaches found names then relations - turns out it works much better the other way around)

Relation text is identified as:
`(Ve (Wo* Pa)?)+`
where
* `Ve = Verb Particle? Adverb?`
* `Wo = Noun | Adjective | Adverb | Pronoun | Determiner`
* `Pa = Preposition or subordinating conjunction | Particle`
* `| =` or, so either of the options
* `? =` optional
* `+ =` at least one, but can be many
* `* =` an arbitrary number of repetitions, including the option for none.

You will need to convert the above rules into a regular expression - this one is harder than the above! You can then run it on a sentence, and for each match identify the named entity to the left and right and create a relation from that.


In [ ]:
def extract(sentence):
    """Given a sentence, as a list of tokens, this returns a list of all relations
    extracted from the sentence. Each relation is a tuple with three entries:
    (named entity one, relation, named entity two)"""
    pos = sentence_pos(sentence) 
    
    str_pos = ''.join(pos)
    
    ner = sentence_ner(sentence, pos)
    
    relation_pattern = '((V(N*|J*|M*|R*|D*)I)?)+'       # pattern for relation
    tag_true = [p for p in re.finditer(relation_pattern, str_pos)]  # find index for patter
    rel_tag = sentence_pos(sentence)  #convert str_pos to sentence for tag_true.start() to tag_true.end()
    
    # Divide the string into two parts: front and rear of relation
    for p in re.finditer(relation_pattern, str_pos):
        for i in range(p.start(), p.end()):
            if i == p.start():
                tag_true_start = i
            if i == p.end():
                tag_true_end = i    
            
    front_str = str_pos[:tag_true[0]]   
    rear_str = str_pos[tag_true[-1]:]
    
    front_sentence = sentence[0:front_str[tag_true_start]]  
    rear_sentence = sentence[front_str[tag_true_end]:]
    
    front_sentence = ' '.join(front_sentence)
    rear_sentence = ' '.join(rear_sentence)
    
    front_pos = sentence_pos(front_sentence)  # pos before tag_true.start()
    rear_pos = sentence_pos(rear_sentence)    # pos after tag_true.end()
    
    front_noun = sentence_ner(front_sentence, front_pos)   # find noun in front 
    rear_noun = sentence_ner(rear_sentence, rear_pos)      # find noun in rear
    
    if front_noun and rear_noun:            # If either one of front and rear not found, discard relation
        ret = [front_noun, rel_tag, rear_noun]
    else:
        ret = [None]
    
    
    return ret


In [ ]:
# Small test of the above...
tests = ['London is full of pigeons.',
         'In 1781 William Herschel discovered Uranus', # 1
         "Trolls really don't like the sun.",
         'Giant owls would enjoy eatting people.',
         "Dragons collect gold, but they don't make microprocessors."] # 2

# 1. Seems to miss William - misclassified it, at least with the model answer.
# 2. Should extract two facts, first sensible, second absurd.

for sentence in tests:
    print(sentence)
    tokens = ogonek.Tokenise(sentence)
    
    rels = extract(tokens[0])
    for rel in rels:
        print('  ' + ' -- '.join(rel))
    print()
    